<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/01_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER
  
## Schema Università

S (<u>Matr</u>,SNome,Citta,ACorso)

D(<u>CD</u>,CNome,Citta)

C(<u>CC</u>,CNome,CD)
- FOREIGN KEY (CD) REFERENCES D(CD)

E(<u>Matr,CC</u>,DATA,VOTO)
- FOREIGN KEY (Matr) REFERENCES S(Matr)
- FOREIGN KEY (CC) REFERENCES C(CC)
 

In [ ]:
pip install duckdb 
!pip install SQLAlchemy==1.4.49

In [ ]:
import pandas as pd

In [ ]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

## Create the the tables
DuckDB

In [ ]:
con.execute("CREATE SCHEMA IF NOT EXISTS uni;")

In [ ]:
con.execute("CREATE TABLE S (\
    Matr VARCHAR(45),\
    SNome VARCHAR(45),\
    Citta VARCHAR(45),\
    ACorso INT,\
    UNIQUE (Matr),\
    PRIMARY KEY (Matr)\
);")

con.execute("CREATE TABLE D(\
 CD VARCHAR(45),\
 CNome VARCHAR(45),\
 Citta VARCHAR(45),\
 UNIQUE (CD),\
 PRIMARY KEY (CD)\
);")

con.execute("CREATE TABLE C(\
 CC VARCHAR(45),\
 CNome VARCHAR(45),\
 CD VARCHAR(45),\
 UNIQUE (CC),\
 PRIMARY KEY (CC)\
);")

con.execute("CREATE TABLE E(\
 Matr VARCHAR(45),\
 CC VARCHAR(45),\
 DATA DATE,\
 VOTO INT,\
 UNIQUE (Matr, CC),\
 PRIMARY KEY (Matr, CC)\
);")


con.execute("INSERT INTO S (Matr, SNome, Citta, ACorso)\
VALUES\
('M1','Lucia Quaranta','SA',1),\
('M2','Giacomo Tedesco','PA',2),\
('M3','Carla Longo','MO',1),\
('M4','Ugo Rossi','MO',1),\
('M5','Valeria Neri','MO',2),\
('M6','Giuseppe Verdi','BO',1),\
('M7','Maria Rossi',null,1);")


con.execute("INSERT INTO D (CD, CNome, Citta)\
VALUES\
('D1','Paolo Rossi','MO'),\
('D2','Maria Pastore','BO'),\
('D3','Paola Caboni','FI');")



con.execute("INSERT INTO C (CC,CNome, CD)\
VALUES\
('C1','Fisica 1','D1'),\
('C2','Analisi Matematica 1','D2'),\
('C3','Fisica 2','D1'),\
('C4','Analisi Matematica 2','D3');")



con.execute("INSERT INTO E (Matr,CC,Data,Voto)\
VALUES\
('M1','C1','1995-06-29',24),\
('M1','C2','1996-08-09',33),\
('M1','C3','1996-03-12',30),\
('M2','C1','1995-06-29',28),\
('M2','C2','1996-07-07',24),\
('M3','C2','1996-07-07',27),\
('M3','C3','1996-11-11',25),\
('M4','C3','1996-11-11',33),\
('M6','C2','1996-01-02',28),\
('M7','C1','1995-06-29',24),\
('M7','C2','1996-04-11',26),\
('M7','C3','1996-06-23',27);")


## Query the DB

In [ ]:
# TEMPLATE QUERY
q = '''
SELECT *
FROM S,C,D,E
'''

con.execute(q) # execute the query
con.fetchdf() # return a pandas dataframe

In [ ]:

# Operatori relazionali : <attr> <op-rel> <cost> 
# dove <op-rel> ∈ {=, <>, >, >=, <, <=}
# 
# Studenti del secondo anno di corso
q = ''' 
SELECT *
FROM S
WHERE ACorso=2;
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Esami con voto compreso tra 24 e 28
q = '''
SELECT *
FROM E
WHERE Voto > 24
AND Voto <= 28;
'''
con.execute(q) 
con.fetchdf()


In [ ]:
# Operatore di set : <attr> IN (<cost1>, ..., <costN>)
#
# Esami con voto pari a 29, 30 oppure con lode (voto pari a 33)
q = '''
SELECT *
FROM E
WHERE Voto IN (29,30,33);
'''
con.execute(q) 
con.fetchdf()


In [ ]:
# Operatore di confronto stringhe : <attr> LIKE <stringa> 
# dove <stringa> puo` contenere i caratteri speciali   (carattere arbitrario) e % (stringa arbitraria)
#
# Studenti il cui nome inizia con A e termina con O

q = '''
SELECT *
FROM S
WHERE SNome LIKE 'V%i'
'''
con.execute(q) 
con.fetchdf()


In [ ]:
# Ordinamento del risultato: ORDER BY <attr> [ASC|DESC]
#
# Studenti di Modena ordinati in senso ascendente rispetto all’anno di corso
q = '''
SELECT Matr,ACorso 
FROM S
WHERE Citta = 'MO' 
ORDER BY ACorso DESC
'''
con.execute(q) 
con.fetchdf()


In [ ]:
# L’ordinamento deve essere fatto rispetto a uno o piu' elementi della <lista-select>: 
# un tale elemento puo' essere indicato anche riportando la sua posizione nella <lista-select>.
#
# Esami del corso C1 ordinati in senso discendente rispetto al voto espresso in sessantesimi, 
# e a parita` di voto rispetto alla matricola

q = '''
SELECT Matr,CC,(60*Voto)/30 
FROM E
WHERE CC='C1'
ORDER BY 3 DESC, Matr DESC
'''

con.execute(q) 
con.fetchdf()


## JOIN operators

In [ ]:
# Patriamo dal prodotto cartesiano: 
# Cioe' la combinazione di tutte le possibili tuple delle tabelle elencate
# nella clausola FROM
q = '''
SELECT *
FROM S,D,E,C
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT * 
FROM S,E
'''

con.execute(q) 
con.fetchdf()

In [ ]:
# Il Join, per definizione e' un "filtro" sul prodotto cartesiano
# cioe', in algebra relazionale, una selezione sul prodotto cartesiano
q = '''
SELECT *
FROM S,E
WHERE S.Matr=E.Matr
'''
con.execute(q) 
con.fetchdf()

# che equivale a:
# q = '''
# SELECT *
# FROM S JOIN E on S.Matr=E.Matr
# '''
# con.execute(q) 
# con.fetchdf()

# q = '''
# SELECT *
# FROM S natural JOIN E
# '''
# con.execute(q) 
# con.fetchdf()

In [ ]:
# Pero' possono esseci studenti che non hanno sostenuto esami.
# Ad esempio:

q = '''
SELECT *
FROM S
WHERE S.Matr not in (SELECT Matr FROM E)
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Se volessimo anche questi studenti nell'output del join
# dobbiamo usare un LEFT join

q = '''
SELECT * FROM S LEFT JOIN E on S.Matr=E.Matr;
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# E se volessi:
# Combinazioni di studenti e di docenti residenti nella stessa citta'
# inclusi gli studenti che risiedono in una citta' 
# che non ha corrispondenza nella relazione dei docenti
q = '''
SELECT *
FROM S JOIN D ON S.Citta=D.Citta;
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# con anche gli studenti di citta' da cui non viene nessun docente:
q = '''
SELECT *
FROM S LEFT JOIN D ON S.Citta=D.Citta;
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Nella clausola FROM e' possibile esprimere anche piu' condizioni di join.
# 
# Ad esempio, se volessimo:
# Per ogni esame con voto superiore a 24 riportare il nome dello studente 
# e il codice del docente del corso
q = '''
SELECT S.SNome,C.CD, E.voto
FROM (S JOIN E ON (S.Matr=E.Matr))
		JOIN C ON (E.CC=C.CC)
WHERE Voto > 24;
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# che equivale a:
q = '''
SELECT S.SNome,C.CD, E.voto
FROM S,E,C
WHERE S.Matr=E.Matr 
AND E.CC=C.CC
AND Voto > 24;
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Con gli outer join, certi attributi vengono duplicati (per definizione)
# Ad sempio:
q = '''
SELECT S.Matr,S.Citta,D.CD, D.Citta
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# come faccio a fare un merge e considerare solo una Citta'?
# funzione: COALESCE(...) sceglie il primo valore non nullo
q = '''
SELECT S.Matr, S.Citta, D.Citta,D.CD
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT S.Matr, COALESCE(S.Citta, D.Citta) AS Citta, D.CD AS CodiceDocente
FROM S LEFT JOIN D ON (S.Citta=D.Citta);
'''
con.execute(q) 
con.fetchdf()

### Self-Join

In [ ]:
# Quando il join avviene tra la tabella e se stessa
# 
# Ad esempio: 
# Selezionare le coppie di studenti della stessa citta'

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta;
'''

con.execute(q) 
con.fetchdf()

In [ ]:
# Cosi' pero' selezionaimo anche uno studente e se stesso
# Quindi modifichiamo aggiungendo:

q = '''
SELECT S1.SNome,S2.SNome,S1.Citta
FROM S S1, S S2
WHERE S1.Citta=S2.Citta
AND S1.Matr < S2.Matr;
'''

con.execute(q) 
con.fetchdf()

#
# DOMANDA:
# perche' <> non andrebbe bene?
#

In [ ]:
# Matricole degli studenti che hanno sostenuto almeno 
# uno degli esami sostenuti dallo studente di nome 'Giuseppe Verdi'

q = '''
SELECT E1.Matr
FROM S, E E1, E E2
WHERE E2.Matr = S.Matr
AND E1.CC = E2.CC
AND S.SNome='Giuseppe Verdi';
'''

con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT E1.Matr
FROM E E1
WHERE E1.CC IN (SELECT E2.CC
				FROM E E2, S
				WHERE S.SNome='Giuseppe Verdi'
				AND S.Matr = E2.Matr);
'''

con.execute(q) 
con.fetchdf()

## Nested Quereis

In [ ]:
# Interrogazioni innestate

# Una interrogazione viene detta innestata o nidificata se la sua condizione
# e' formulata usando il risultato di un’altra interrogazione, 
# chiamata subquery

q = '''
SELECT *
FROM S
WHERE S.Matr IN (SELECT Matr
				 FROM E);
'''

con.execute(q) 
con.fetchdf()

## ANY/IN/EXISTS
- Operatori quantificati: `<attr><op-rel>[ANY|ALL]<subquery`
- Operatore di set: `<attr> [NOT] IN <subquery>`
- Quantificatore esistenziale : `[NOT] EXISTS <subquery>`

In [ ]:
#  TIPO [ANY/ALL]
# Per esprimere una condizione che e' vera quando 
# qualsiasi (ANY) opppure tutti (ALL) gli elementi
# restituiti dalla subquery fanno verificare la condizione
# i.e., condizione risulta TRUE
 
# ANY
# Nome degli studenti che hanno sostenuto l’esame del corso C1

q = '''
SELECT SNome
FROM S
WHERE Matr = ANY ( SELECT Matr
                        FROM   E
                        WHERE  CC='C1');
'''
con.execute(q) 
con.fetchdf()


In [ ]:
q = '''
SELECT SNome
FROM S natural JOIN E
WHERE E.CC = 'C1';
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# ALL
# Studenti con anno di corso piu` basso
q = '''
SELECT *
FROM S
WHERE ACorso <= ALL ( SELECT ACorso
					  FROM S);
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Spesso possono esserci diversi modi di esprimere la stesa query
# es: Nome degli studenti che hanno sostenuto l’esame del corso C1
# senza subquery:
q = '''
SELECT S.SNome
FROM E join S on E.Matr=S.Matr
WHERE E.CC = 'C1';
'''
con.execute(q) 
con.fetchdf()

# # oppure:
# q = '''
# SELECT S.SNome
# FROM S
# WHERE S.Matr in (SELECT E.Matr 
# 				 FROM E WHERE E.CC='C1');
# '''
# con.execute(q) 
# con.fetchdf()

In [ ]:
# ALTRO ESEMPIO:
# Nome degli studenti che hanno sostenuto l’esame di un corso del docente D1
#/
q = '''
SELECT SNome
FROM E,S,C
WHERE E.Matr=S.Matr 
AND E.CC=C.CC
AND CD='D1';
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# che puo' essere espressa anche come:
q = '''
SELECT SNome
FROM S
WHERE S.Matr IN (SELECT E.Matr 
				 FROM E 
				 WHERE E.CC IN (SELECT C.CC 
				 				FROM C 
				 				WHERE C.CD='D1'));
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Provare la precedente query partendo dalla subquery piu' interna

# Fino ad ora abbiamo considerato delle subquery dove
# la query innestata era indipendente dalla query esterna

# Se vogliamo pero' selezionare:
# Per ogni citta', il nome degli studenti con anno di corso piu` alto
q = '''
SELECT S1.Citta, S1.SNome, S1.ACorso
FROM S S1
WHERE S1.ACorso >= ALL (SELECT S2.ACorso
					  	FROM S S2
					  	WHERE S1.Citta=S2.Citta);
'''
con.execute(q) 
con.fetchdf()
# S1 e' la tupla corrente

### EXISTS

In [ ]:
# QUALIFICATORE ESISTENZIALE:

# EXISTS

# Mettendo in relazione subquery e query estenza con una clausola
# che è vera solo se per la tupla corrente il risultato della subquery
# restituisce qualcosa (i.e., non è nullo).
# Da qui: "EXISTS", cioè ESISTE qualcosa che rende TRUE quel predicato.

# Ad esempio:
# Nome degli studenti che hanno sostenuto l’esame del corso C1:

q = '''
SELECT SNome
FROM S
WHERE EXISTS (SELECT *
			  FROM E
			  WHERE E.Matr=S.Matr
			  AND E.CC = 'C1');
'''
con.execute(q) 
con.fetchdf()			  

In [ ]:
# Allo stesso modo, il negato della clausola EXISTS:

# NOT EXISTS (<subquery>)

# ha valore true se e solo se l’insieme di valori restituiti da <subquery> e' vuoto.

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1
q = '''
SELECT SNome
FROM S
WHERE NOT EXISTS ( SELECT *
				   FROM E
				   WHERE E.Matr=S.Matr 
				   AND E.CC='C1' )
'''
con.execute(q) 
con.fetchdf()

# Tip:
# Se state usando EXISTS senza una subquery crrelata,
# molto probabilmente state commettendo un errore:

# Perche'?
# (hint: cosa restituisce la subquery?) 

### Riduzione di query innestate

In [ ]:
# Le query innestate formulate con i seguenti operatori 
# si possono ridurre a query di join equivalenti 
# (stessa risposta per ogni possibile istanza della base di dati):
# • IN
# • ANY (con qualsiasi operatore di confronto) 
# • EXISTS con subquery correlata
# 

				   
# Nome degli studenti che hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr IN ( SELECT Matr
				FROM E
				WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 2. 
q = '''
SELECT SNome 
FROM S
WHERE Matr =ANY ( SELECT Matr
				  FROM E
				  WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 3.
q = '''
SELECT SNome 
FROM S
WHERE EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
con.execute(q) 
con.fetchdf()

			   
# QUERY EQUIVALENTE CON JOIN
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC='C1'
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# INVECE:
 
# Le query innestate formulate con i seguenti operatori 
# non si possono ridurre:

# • NOT IN
# • ALL (con qualsiasi operatore di confronto) 
# • NOT EXISTS con subquery correlata

# Ad esempio:
# Nome degli studenti che non hanno sostenuto l’esame del corso C1

# 1. 
q = '''
SELECT SNome 
FROM S
WHERE Matr NOT IN ( SELECT Matr
	 				FROM E
					WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 2.
q = '''
SELECT SNome 
FROM S
WHERE Matr <>ALL ( SELECT Matr
				  FROM E
				  WHERE CC='C1')
'''
con.execute(q) 
con.fetchdf()

# 3.
q = '''
SELECT SNome 
FROM S
WHERE NOT EXISTS ( SELECT *
			   FROM E
			   WHERE E.Matr=S.Matr
			   AND CC='C1')
'''
con.execute(q) 
con.fetchdf()

# NON SI POSSONO ESPRIMRE CON JOIN

In [ ]:
# NOTARE: la differenza con la seguente query
# ERRATA (per la consegna):
q = '''
SELECT SNome
FROM E,S
WHERE E.Matr=S.Matr 
AND E.CC <> 'C1'
'''
con.execute(q) 
con.fetchdf()
# cosa restituisce?

## Funzioni di Aggregazione

In [ ]:
# 		Funzioni aggregate 
# 		(column functions)

# Dato un insieme di tuple che soddisfano un predicato
# (e.g., il risultato di una query qualsiasi di quelle
# viste finora)
# le funzione di aggregazione restituiscono un risultato
# "aggregato" calcolato su queell'insieme.

# Possono essere cosi' facilmente ritrovati:
# La media dei voti di uno studete;
# Il voto massimo in un dato esame;
# Il numero di studenti che proviene da una data citta;
# ... 

q = '''
SELECT COUNT(*)
FROM S
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT AVG(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT MAX(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()


In [ ]:
q = '''
SELECT MIN(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT SUM(E.VOTO)/COUNT(E.VOTO)
FROM E
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT DISTINCT(E.CC)
FROM E
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT E.CC
FROM E
'''
con.execute(q) 
con.fetchdf()

In [ ]:
q = '''
SELECT COUNT(DISTINCT(E.CC))
FROM E
'''
# Alcuni DBMS non permettono questa sintassi
con.execute(q) 
con.fetchdf()

In [ ]:
# Es. “Numero di studenti presenti” 
q = '''
SELECT COUNT(Matr)
FROM S
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Es. “Numero di studenti che hanno sostenuto almeno un esame” 
q = '''
SELECT COUNT(DISTINCT Matr)
FROM E
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Es. “Numero di studenti con anno di corso non nullo” 
q = '''
SELECT COUNT(ACorso)
FROM S
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Es. “Numero di anni di corso di studenti presenti” 
q = '''
SELECT COUNT(DISTINCT ACorso)
FROM S
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Es. “Numero di coppie distinte matricola-voto” 
q = '''
SELECT COUNT(DISTINCT Matr, Voto)
FROM E
'''
# Alcuni DBML non permettono questa sintassi
con.execute(q) 
con.fetchdf()

In [ ]:
# Es. “Voto medio degli esami sostenuti dalla matricola M1”
q = '''
SELECT AVG(Voto)
FROM E 
WHERE Matr='M1'
'''
con.execute(q) 
con.fetchdf()


# che e' equivalente a
q = '''
SELECT SUM(Voto)/COUNT(Voto) 
FROM E 
WHERE Matr='M1'
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Es. “Studenti il cui anno di corso `e minore di quello massimo presente”
q = '''
SELECT *
FROM S
WHERE ACorso < ( SELECT MAX(ACorso)
				 FROM S)
'''
con.execute(q) 
con.fetchdf()

In [ ]:
# Considerate la sequente query, con funzione di aggregazione:

q = '''
SELECT Matr,MAX(Voto)
FROM E
'''
con.execute(q) 
con.fetchdf()
# PERCHE' E' SBAGLIATA?

